## Define functions to query OMOP database

In [ ]:
library(dplyr)
library(RSQLite)

con <- dbConnect(SQLite(), "OMOP.db")

#' convert icd codes (ICD9/10) to OMOP concept ids
icd_to_omop <- function(icd, data.frame = FALSE){
    
    icd2omop <- tbl(con, "icd2omop")

    # convert ICD9/10 to OMOP ids
    omop.df <- icd2omop %>%
      filter(icd_code %in% !!icd) %>%
      collect() 
    
    if (data.frame) omop.df else omop.df$omop_concept_id
    
}

#' find descendant concept_ids for given omop condition concept_ids
omop_condition_descendants <- function(omop) {
    
    pd <- tbl(con, "condition_descendants")
    
    res <- pd |>
        filter(ancestor_concept_id %in% !!omop) |>
        select(descendant_concept_id) |>
        collect() |>
        pull(descendant_concept_id)
    
    res
    
}

#' create data frame with (person_id, min_date, max_date, n_dates) for given omop concept_ids
#' If descendants = TRUE, also include descendant concept_ids
omop_condition_occurrence <- function(omop, descendants = TRUE) {
    
    co <- tbl(con, "condition_occurrence")
    demo <- tbl(con, "demographics")
    
    keys <- omop
    
    if (descendants) {
        keys <- omop_condition_descendants(keys)
    }
    
    res <- co |> 
      filter(condition_concept_id %in% !!keys) |> 
      select(person_id, condition_start_date) |>
      group_by(person_id, condition_start_date) |>
          summarize(n = 1, .groups = "drop") |>
      group_by(person_id) |>
          summarize(min_date = min(condition_start_date, na.rm = TRUE),
                    max_date = max(condition_start_date, na.rm = TRUE),
                    n_dates = n(), .groups = "drop") |> 
      left_join(select(demo, person_id, dob), by = 'person_id') |>
      mutate(min_age = (min_date - dob)/365.25, 
             max_age = (max_date - dob)/365.25) |> 
      select(person_id, min_date, max_date, min_age, max_age, n_dates) |>
      collect() |>
      mutate(min_date = as.Date(min_date),
             max_date = as.Date(max_date))
    
    res
}

#' find descendant concept_ids for given omop procedure concept_ids
omop_procedure_descendants <- function(omop) {
    
    pd <- tbl(con, "procedure_descendants")
    
    res <- pd |>
        filter(ancestor_concept_id %in% !!omop) |>
        select(descendant_concept_id) |>
        collect() |>
        pull(descendant_concept_id)
    
    res
    
}

#' create data frame with (person_id, min_date, max_date, n_dates) for given omop concept_ids
#' If descendants = TRUE, also include descendant concept_ids
omop_procedure_occurrence <- function(omop, descendants = TRUE) {
    
    co <- tbl(con, "procedure_occurrence")
    demo <- tbl(con, "demographics")
    
    keys <- omop
    
    if (descendants) {
        keys <- omop_procedure_descendants(keys)
    }
    
    res <- co |> 
      filter(procedure_concept_id %in% !!keys) |> 
      select(person_id, procedure_date) |>
      group_by(person_id, procedure_date) |>
         summarize(n = 1, .groups = "drop") |>
      group_by(person_id) |>
      summarize(min_date = min(procedure_date, na.rm = TRUE),
                max_date = max(procedure_date, na.rm = TRUE),
                n_dates = n(), .groups = "drop") |>
      left_join(select(demo, person_id, dob), by = 'person_id') |>
      mutate(min_age = (min_date - dob)/365.25, 
             max_age = (max_date - dob)/365.25) |> 
      select(person_id, min_date, max_date, min_age, max_age, n_dates) |>
      collect() |>
      mutate(min_date = as.Date(min_date),
             max_date = as.Date(max_date))
    
    res
}

#' retrieve person_ids that form the universe for case-control phenotypes
#' i.e. sex_at_birth in (Male, Female), has_srwgs, and has_ehr_data
get_universe_cc <- function() {
    
    demo <- tbl(con, "demographics")
    
    res <- demo |>
          filter(sex_at_birth %in% c('Male', 'Female')) |>
          filter(has_srwgs == 1 & has_ehr_data == 1) |>
          select(person_id) |>
          collect() |>
          pull(person_id)
}

#' return cases and controls for given condition and procedure concept ids
#' this is the workhorse function
#' it automatically includes descendants of all concept_ids
get_cc_lists <- function(universe, case.cond = c(), case.proc = c(), 
                           case.excl.cond = c(), case.excl.proc = c(),
                           ctrl.excl.cond = c(), ctrl.excl.proc = c()){
    
    
    cases.0 <- intersect(universe, 
                         union(omop_condition_occurrence(case.cond)$person_id, 
                               omop_procedure_occurrence(case.proc)$person_id))
    controls.0 <- setdiff(universe, cases.0) 

    cases.excl <- union(omop_condition_occurrence(case.excl.cond)$person_id,
                        omop_procedure_occurrence(case.excl.proc)$person_id)
    cases <- setdiff(cases.0, cases.excl)


    controls.excl <- union(omop_condition_occurrence(ctrl.excl.cond)$person_id,
                           omop_procedure_occurrence(ctrl.excl.proc)$person_id)
    controls <- setdiff(controls.0, controls.excl)
    
    list(cases = cases, 
         controls = controls)
    
}



get_cc_lists2 <- function(universe, 
                          case.cond = c(), 
                          case.proc = c(),
                          case.min.age = 0,
                          case.max.age = Inf,
                          case.excl.cond = c(), 
                          case.excl.proc = c(),
                          ctrl.excl.cond = c(), 
                          ctrl.excl.proc = c()){
    
    
    co <- omop_condition_occurrence(case.cond) 
    po <- omop_procedure_occurrence(case.proc)
    
    # cases.0 (no age considered) only used to find controls.0
    cases.0 <- intersect(universe, 
                         union(co$person_id, 
                               po$person_id))
    controls.0 <- setdiff(universe, cases.0) 

    # filter cases on case.min.age <= age_at_first_diagnosis <= case.max.age
    copo <- rbind(co, po) |>
            group_by(person_id) |>
            summarize(age_at_first_diagnosis = min(min_age), 
                      .group = 'drop') |>
            filter(case.min.age <= age_at_first_diagnosis & age_at_first_diagnosis <= case.max.age)
    
    cases.1 <- intersect(universe, copo$person_id)
    
    cases.excl <- union(omop_condition_occurrence(case.excl.cond)$person_id,
                        omop_procedure_occurrence(case.excl.proc)$person_id)
    cases <- setdiff(cases.1, cases.excl)


    controls.excl <- union(omop_condition_occurrence(ctrl.excl.cond)$person_id,
                           omop_procedure_occurrence(ctrl.excl.proc)$person_id)
    controls <- setdiff(controls.0, controls.excl)
    
    list(cases = cases, 
         controls = controls)
    
}





#' add phenotype from case-control lists `pheno` to data frame `df`
add_pheno <- function(df, pheno, label) {
    
    df |>
        mutate(!!sym(label) := case_when(
              person_id %in% pheno$cases    ~ 1L,
              person_id %in% pheno$controls ~ 0L,
              TRUE                          ~ NA_integer_
    ))
}

## Example usage

Start with definition of phenotype

In [2]:
case.cond.icd = c('J41','J42')
case.proc = c(2105103, 2774098, 2774104)

case.excl.cond.icd = c('J43','J44')

The ICD codes need to be converted to OMOP concept ids

In [3]:
case.cond <- icd_to_omop(case.cond.icd)
case.excl.cond <- icd_to_omop(case.excl.cond.icd)

We can also check the conversion by returning a data frame

In [4]:
icd_to_omop(case.cond.icd, data.frame = TRUE)

icd_concept_id,icd_code,icd_name,icd_vocabulary,omop_concept_id,source_vocabulary,source_code,source_name
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
1569485,J41,Simple and mucopurulent chronic bronchitis,ICD10CM,255841,SNOMED,63480004,Chronic bronchitis
35208017,J42,Unspecified chronic bronchitis,ICD10CM,255841,SNOMED,63480004,Chronic bronchitis


In [5]:
universe <- get_universe_cc()

The next query takes a rather long time the first time it is run. This is due to database warm-up. The time can be improved by using a SSD disk instead of a regular spinning hard disk.

Subsequent queries should only take around 5 seconds.

In [7]:
ph <- get_cc_lists(universe, 
                   case.cond = case.cond, 
                   case.proc = case.proc, 
                   case.excl.cond = case.excl.cond)

Create a tibble with person_ids from the universe, and add phenotype using the cc list above

In [8]:
pheno <- tibble(person_id = universe) 
pheno <- add_pheno(pheno, ph, "myphenotype")

In [9]:
table(pheno$myphenotype, useNA = "ifany")


     0      1   <NA> 
302750   6325   5545 

In [ ]:
head(pheno)

One can add multiple phenotypes iteratively. To convert the pheno df to a format that can be used with regenie, we need to add the FID and IID columns

In [11]:
regenie <- pheno |>
  select(FID = person_id, IID = person_id, everything())

In [ ]:
head(regenie)

In [ ]:
require(data.table)
fwrite(regenie, "regenie_pheno.tsv", sep = "\t", na = "NA")

get_cc_lists2 also allows specification of minimum or maximum age at first diagnosis

In [13]:
ph2 <- get_cc_lists2(universe, 
                   case.cond = case.cond, 
                   case.proc = case.proc, 
                   case.min.age = 65,
                   case.excl.cond = case.excl.cond)

In [14]:
pheno <- add_pheno(pheno, ph2, "ph2_age")

In [18]:
table(pheno$myphenotype, pheno$ph2_age, useNA = "ifany")

      
            0      1   <NA>
  0    302750      0      0
  1         0   2579   3746
  <NA>      0      0   5545